In [ ]:
# read in the text 
# may want to remove opening and closing of books but may be inconsistent
import pandas as pd 
import csv

book1 = open("alice_wonderland.txt", "r")
book1 = book1.read()

book2 = open("dolls_house.txt", "r")
book2 = book2.read()

book3 = open("dracula.txt", "r")
book3 = book3.read()

book4 = open("emma.txt", "r")
book4 = book4.read()

book5 = open("two_cities.txt", "r")
book5 = book5.read()

print(book5[:100])



In [17]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [24]:
# clean the text - lowercase, remove stopwords, remove special chars, split into sentences and remove v short sentences
import re

stops = set(stopwords.words('english'))

raw_text = " ".join([book1, book2, book3, book4, book5])
text = raw_text.lower()
text = re.sub("[^A-Za-z’'.]", " ", text)

text_cleaned = " ".join([i for i in text.split(" ") if i not in stops])
text_cleaned = re.sub("[ ]+", " ", text_cleaned)

# removing very short sentences
sentences = text_cleaned.split(".")
sentences = [i.strip() for i in sentences if len(i.strip().split(" ")) > 2]

print(sentences[:10])




['alice’s adventures wonderland lewis carroll millennium fulcrum edition', 'rabbit hole alice beginning get tired sitting sister bank nothing twice peeped book sister reading pictures conversations use book ’ thought alice without pictures conversations ’ considering mind well could hot day made feel sleepy stupid whether pleasure making daisy chain would worth trouble getting picking daisies suddenly white rabbit pink eyes ran close her', 'nothing remarkable alice think much way hear rabbit say oh dear oh dear shall late ’ thought afterwards occurred ought wondered time seemed quite natural rabbit actually took watch waistcoat pocket looked hurried alice started feet flashed across mind never seen rabbit either waistcoat pocket watch take burning curiosity ran across field fortunately time see pop large rabbit hole hedge', 'another moment went alice never considering world get again', 'rabbit hole went straight like tunnel way dipped suddenly suddenly alice moment think stopping found

In [37]:
# identify phrases that might skew (eg one meaning across two words not two strongly related words)
from gensim.models.phrases import Phrases, Phraser

words_list = [i.split() for i in sentences]

phrases = Phrases(words_list, min_count=30, progress_per=10000)
print(words_list)

[['alice’s', 'adventures', 'wonderland', 'lewis', 'carroll', 'millennium', 'fulcrum', 'edition'], ['rabbit', 'hole', 'alice', 'beginning', 'get', 'tired', 'sitting', 'sister', 'bank', 'nothing', 'twice', 'peeped', 'book', 'sister', 'reading', 'pictures', 'conversations', 'use', 'book', '’', 'thought', 'alice', 'without', 'pictures', 'conversations', '’', 'considering', 'mind', 'well', 'could', 'hot', 'day', 'made', 'feel', 'sleepy', 'stupid', 'whether', 'pleasure', 'making', 'daisy', 'chain', 'would', 'worth', 'trouble', 'getting', 'picking', 'daisies', 'suddenly', 'white', 'rabbit', 'pink', 'eyes', 'ran', 'close', 'her'], ['nothing', 'remarkable', 'alice', 'think', 'much', 'way', 'hear', 'rabbit', 'say', 'oh', 'dear', 'oh', 'dear', 'shall', 'late', '’', 'thought', 'afterwards', 'occurred', 'ought', 'wondered', 'time', 'seemed', 'quite', 'natural', 'rabbit', 'actually', 'took', 'watch', 'waistcoat', 'pocket', 'looked', 'hurried', 'alice', 'started', 'feet', 'flashed', 'across', 'mind',

In [38]:
bigram = Phraser(phrases)
sents = bigram[words_list]

In [ ]:
# from collections import defaultdict
# word_freq = defaultdict(int)
# for sent in sents:
#     for i in sent:
#         word_freq[i] += 1
# len(word_freq)
# sorted(word_freq, key=word_freq.get, reverse=True)[:10]

In [40]:
import multiprocessing
from gensim.models import Word2Vec 


In [42]:
cores = multiprocessing.cpu_count()
cores

2

In [47]:
wtv_model = Word2Vec(min_count=10,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [48]:
from time import time

t = time()

wtv_model.build_vocab(sents, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

Time to build vocab: 0.03 mins


In [49]:
from time import time
t = time()

wtv_model.train(sents, total_examples=wtv_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

Time to train the model: 0.64 mins


In [50]:
wtv_model.init_sims(replace=True)

In [ ]:
# note, so far have neither stemmed or lemmatized which should do and remake model, as well as adding more data
print(wtv_model.most_similar('alice'))